In [ ]:
!pip install PyMuPDF

     |████████████████████████████████| 8.8 MB 4.3 MB/s 


In [ ]:
!pip install langdetect

     |████████████████████████████████| 981 kB 5.1 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=0d4fa12ced0d7de329a7b47d320c28fe923f2d0a421053411ce222aebd4d8468
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect


In [ ]:
!pip install html2markdown

  Created wheel for html2markdown: filename=html2markdown-0.1.7-py3-none-any.whl size=5255 sha256=4939bf86577dca6932426d04e28f973758afa1ddd824704b1591c9b77f47c176
  Stored in directory: /root/.cache/pip/wheels/4e/ab/65/600034392f01945283479f3886b57e364dd2b0ad0220a977b7
Successfully built html2markdown


In [ ]:
!pip install markdownify

     |████████████████████████████████| 97 kB 3.3 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import difflib
import fitz
from pprint import pprint
import sys
#from fpdf import FPDF
from langdetect import detect
import html2markdown
import re
import os
from markdownify import markdownify
import markdown
from bs4 import BeautifulSoup # pip install beautifulsoup4
import json

def convert_to_html(doc):
    text = ""
    for page in doc.pages():
        for line in page.get_text("html").splitlines():
            if (line[0]+line[1]) in ['<p','<d']:
                text+=line
                #text+='\n'
    return text

def create_html_files():
    files = os.listdir('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/pdf/')
    platforms = []
    companies = []
    for f in files:
        if '.pdf' not in f:
            companies.append(f)

    for c in companies:
        pols = os.listdir('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/pdf/' + c + '/')
        for p in pols:
          print(c+"/"+p)
          dates = os.listdir('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/pdf/' + c + '/' + p + '/')
          ds = []
          for d in dates:
            print(d)
            doc = fitz.open('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/pdf/' + c + '/' + p + '/'+d)
            t = convert_to_html(doc)
            name = d.split('.')[0]
            text_file = open("/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/html/"+c+"/"+p+"/"+name+".html", "w")
            text_file.write(t)
            text_file.close()

def create_markdown_files():
    files = os.listdir('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/html/')
    platforms = []
    companies = []
    for f in files:
        if '.pdf' not in f:
            companies.append(f)

    for c in companies:
        pols = os.listdir('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/html/' + c + '/')
        for p in pols:
            dates = os.listdir('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/html/' + c + '/' + p + '/')
            ds = []
            for d in dates:
                file = open('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/html/' + c + '/' + p + '/'+d, "r").read()
                t = markdownify(file, heading_style="ATX")
                try:
                    if detect(t) != 'en':
                        continue
                except:
                    continue
                name = d.split('.')[0]
                text_file = open("/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/markdown/"+c+"/"+p+"/"+name+".md", "w")
                text_file.write(t)
                text_file.close()
def add_color(c, text):
    if c=="red":
        return '<span style="color: red;">'+text+'</span>'
    else:
        return '<span style="color: green;">'+text+'</span>'
    
def get_diff_file(a,b):
    d = difflib.Differ()
    delta = list(d.compare(a, b))
    if len(delta)<=1:
        print("pow")
        return -1
    diff_list = get_list_of_diff_sequences(delta)
    file = ""
    for e in diff_list:
        #print(e)
        file += e
    return file

def diffcheck_all_mds():
    files = os.listdir('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/markdown/')
    platforms = []
    companies = []
    for f in files:
        if '.pdf' not in f:
            companies.append(f)
    for c in companies:
        pols = os.listdir('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/markdown/' + c + '/')
        for p in pols:
            dates = os.listdir('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/markdown/' + c + '/' + p + '/')
            for i in range(0,len(dates)-1):
                file1 = open('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/markdown/' + c + '/' + p + '/'+dates[i], "r").read()
                file2 = open('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/markdown/' + c + '/' + p + '/'+dates[i+1], "r").read()
                diff_file = get_diff_file(file1,file2)
                if diff_file == -1:
                    continue
                #return diff_file
                name = dates[i].split('.')[0] + "_vs_" + dates[i+1].split('.')[0]
                text_file = open("data/diffs/"+name+".md", "w")
                #print(text_file)
                text_file.write(diff_file)
                text_file.close()
                with open("data/diffs/"+name+".md", 'r') as f:
                    text = f.read()
                    html = markdown.markdown(text)
                with open("data/diffs/"+c+'/'+p+'/'+name+".html", 'w') as f:
                    f.write(html)
                print(name)
            
def get_list_of_diff_sequences(delta):
    diff_list = []
    current_sign = ' '
    sequence = ""
    #print(len(delta))
    i=0
    while i<len(delta)-1:
        if delta[i][0] == '-':
            counter = i
            sequence +='<span style="background-color: red;">'
            while delta[counter][0] == '-' and counter <len(delta)-1:
                sequence+=delta[counter][2]
                counter +=1
                i+=1
            sequence +='</span>'
            diff_list.append(sequence)
            sequence = ""
            i=counter-1
            #print(counter)
        elif delta[i][0] == '+':
            counter = i
            sequence +='<span style="background-color: green;">'
            while delta[counter][0] == '+' and counter <len(delta)-1:
                sequence+=delta[counter][2]
                counter +=1
            sequence +='</span>'
            diff_list.append(sequence)
            sequence = ""
            i=counter-1
            #print(counter)
        elif delta[i][0] == ' ':
            counter = i
            while delta[counter][0] == ' ' and counter <len(delta)-1:
                #print(delta[counter] + "///"+str(counter))
                sequence+=delta[counter][2]
                counter +=1
            diff_list.append(sequence)
            sequence = ""
            i=counter-1
            #print(counter)
        i+=1
        #print(i)
    return diff_list

    
def merge_html_files():
    files = os.listdir('./data/diffs/')
    platforms = []
    companies = []
    for f in files:
        if '.md' not in f and '.html' not in f and f!='Total diffs':
            companies.append(f)
    for c in companies:
        pols = os.listdir('./data/diffs/' + c + '/')
        for p in pols:
            total_file = ""
            dates = os.listdir('./data/diffs/' + c + '/' + p + '/')
            for dat in dates:
                if '.md' in dat:
                    dates.remove(dat)
            for d in dates:
                with open('./data/diffs/'+ c + '/'+ p +'/'+ d, 'r') as f:
                    name_arr = d.split('_')
                    print(name_arr)
                    headline = name_arr[0] + '_vs_' + name_arr[4]
                    text = f.read()
                    total_file+='<p><h1>'+headline+'</h1></p>'
                    total_file +=text
            with open('./data/diffs/Total diffs/'+ c + '/'+c+'_'+ p+'_total_diff.html', 'w') as total:
                total.write(total_file)
def delete_html_and_md_files():
    files = os.listdir('./data/unique/html/')
    platforms = []
    companies = []
    for f in files:
        if '.pdf' not in f:
            companies.append(f)
    for c in companies:
        pols = os.listdir('./data/unique/html/' + c + '/')
        for p in pols:
            dates = os.listdir('./data/unique/html/' + c + '/' + p + '/')
            ds = []
            for d in dates:
                name = d.split('.')[0]
                #file = open('./data/files/html/' + c + '/' + p + '/'+d, "r").read()
                try:
                    os.remove('./data/unique/html/' + c + '/' + p + '/'+name+'.html')
                    os.remove('./data/unique/markdown/' + c + '/' + p + '/'+name+'.md')
                except:
                    print(".")


def md_to_text(md):
    html = markdown.markdown(md)
    soup = BeautifulSoup(html, features='html.parser')
    return soup.get_text()

def create_diff_view_files():
    platforms = []
    companies = os.listdir('./data/unique/markdown/')
    for c in companies:
        pols = os.listdir('./data/unique/markdown/' + c + '/')
        for p in pols:
            files = os.listdir('./data/unique/markdown/' + c + '/' + p + '/')
            for file in files:
                file1 = open('./data/unique/markdown/' + c + '/' + p + '/'+file, "r").read()
                with open('./data/unique/diffView/' + c + '/' + p + '/'+file, 'w') as f:
                    f.write(md_to_text(file1))

                    


def full_pipeline():
    create_html_files()
    print("Created html files")
    create_markdown_files()
    print("Created md files")
    #create_diff_view_files()
    print("Created diff_view_files")
    #diffcheck_all_mds()
    print("fin")

ModuleNotFoundError: ignored

In [ ]:
import os
import json
import numpy as np

In [ ]:
def create_platforms_json(basepath = '/content/drive/MyDrive/Code/HIIG/pgArchive'):
    files = os.listdir(basepath+'/data/unique/diffView/')
    platforms = []
    companies = []
    for f in files:
      if '.pdf' not in f:
            companies.append(f)
    for c in companies:
        c_dict = {}
        c_dict["name"] = c
        c_dict["slug"] = c.lower()
        c_dict["policies"] = []
        pols = os.listdir(basepath+'/data/unique/pdf/' + c + '/')
        for p in pols:
            slug = ''
            if p == 'Terms of Service':
                slug = 'ts'
            if p == 'Community Guidelines':
                slug = 'cg'
            if p == 'Privacy Policy':
                slug = 'pp'
            diffchecks = get_diffchecks(c,p) #list of json objects {date1, date2, html}
            dates = os.listdir(basepath+'/data/unique/markdown/' + c + '/' + p + '/')
            ds = []
            for d in dates:
                ds.append(d.split('_')[0])
                #if len(ds)>=8:
                #    break
            ds = list(np.sort(ds))
            p_dict = {'name': p, 'slug': slug, 'dates': ds, 'diffchecks': diffchecks}
            c_dict["policies"].append(p_dict)
        platforms.append(c_dict)
    with open(basepath+'/data/json/platforms.json', 'w') as fi:
        json.dump(platforms, fi)
    print("Platform Overview Created...")

import numpy as np

def get_diffchecks(c,p, basepath = '/content/drive/MyDrive/Code/HIIG/pgArchive'):
    diffs = os.listdir(basepath+'/data/unique/markdown/' + c + '/' + p + '/')
    diffs = np.sort(diffs)
    output = []
    print(c+' | '+p)
    for i in range(0,len(diffs)-1):
        d1 = diffs[i]
        d2 = diffs[i+1]
        date1 = d1.split('_')[0]
        date2 = d2.split('_')[0]
        print(date2)
        with open(basepath+'/data/unique/markdown/'+c+'/'+p+'/'+d1, 'r') as f1:
            md1 = f1.read()
        with open(basepath+'/data/unique/markdown/'+c+'/'+p+'/'+d2, 'r') as f2:
            md2 = f2.read()
        dat = {'dateOld': date1, 'dateNew':date2, 'mdOld': md1, 'mdNew':md2}
        output.append(dat)
        #if len(output)>=8:
         #break
    return output                    

In [ ]:
files_total = os.listdir('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique')
for f in files_total:
  if '.ipynb' in f:
    continue
  if os.path.isdir('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/'+f):
    companies = os.listdir('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/'+f)
    for c in companies:
      policies = os.listdir('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/'+f+'/'+c)
      for p in policies:
        files = os.listdir('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/'+f+'/'+c+'/'+p)
        for fi in files:
          print(fi)
          if len(fi.split('_')) == 2:
            date = fi.split('_')[0]
            ending = ''
            if f == 'pdf':
              ending='.pdf'
            if f == 'html':
              ending = '.html'
            if f == 'markdown':
              ending = '.md'
            if f == 'diffView':
              ending = '.md'
            new_name = date+'_'+c+'_'+p+ending
            os.rename('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/'+f+'/'+c+'/'+p+'/'+fi, 
                      '/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/'+f+'/'+c+'/'+p+'/'+new_name)
          else:
            date = fi.split('_')[0]
            if len(date) > 8:
              date = date[:8]
              new_name = date+'_'+fi.split('_')[1]+'_'+fi.split('_')[2]
              print(new_name)
              os.rename('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/'+f+'/'+c+'/'+p+'/'+fi, 
                        '/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/'+f+'/'+c+'/'+p+'/'+new_name)
            elif ' ' in fi:
              new_name = fi.replace(' ', '')
              os.rename('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/'+f+'/'+c+'/'+p+'/'+fi, 
                        '/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/'+f+'/'+c+'/'+p+'/'+new_name)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<h1>1. Convert pdfs to text</h1>

In [ ]:
# full_pipeline()

20200227_Facebook_CommunityGuidelines.pdf
20200827_Facebook_CommunityGuidelines.pdf
20200528_Facebook_CommunityGuidelines.pdf
20200504_Facebook_CommunityGuidelines.pdf
20200130_Facebook_CommunityGuidelines.pdf
20190903_Facebook_CommunityGuidelines.pdf
20190827_Facebook_CommunityGuidelines.pdf
20190730_Facebook_CommunityGuidelines.pdf
20190724_Facebook_CommunityGuidelines.pdf
20190701_Facebook_CommunityGuidelines.pdf
20190426_Facebook_CommunityGuidelines.pdf
20190320_Facebook_CommunityGuidelines.pdf
20191216_Facebook_CommunityGuidelines.pdf
20190929_Facebook_CommunityGuidelines.pdf
20190927_Facebook_CommunityGuidelines.pdf
20190924_Facebook_CommunityGuidelines.pdf
Created html files
Created md files
Created diff_view_files
fin


<h1>2. Create Json Object for pga </h1>

In [ ]:
# files_total = os.listdir('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique')
# for f in files_total:
#   if '.ipynb' in f:
#     continue
#   if os.path.isdir('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/'+f):
#     companies = os.listdir('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/'+f)
#     for c in companies:
#       policies = os.listdir('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/'+f+'/'+c)
#       for p in policies:
#         files = os.listdir('/content/drive/MyDrive/Code/HIIG/pgArchive/data/unique/'+f+'/'+c+'/'+p)
#         for fi in files:
#           if ' ' in fi:


In [ ]:
create_platforms_json()

YouTube | Terms of Service
20050807
20051021
20051028
20070926
20090320
20100618
20150109
20150507
20180602
20200104
20201121
20210320
20211127
YouTube | Privacy Policy
20051031
20081120
20090320
20101217
20120727
20130628
20131220
20140331
20141219
20150303
20150506
20150605
20150705
20150820
20160707
20160829
20170307
20171008
20171218
20180601
20190126
20191026
20200404
20200704
20200829
20201003
20210206
20210703
Twitter | Terms of Service
20080806
20090429
20090918
20101121
20110518
20110612
20120520
20130119
20140925
20150524
20160201
20161005
20171009
20180427
20180504
20180526
20190819
20200106
20200619
20210825
Twitter | Privacy Policy
20090429
20091230
20100312
20100611
20101121
20110628
20120520
20130704
20131024
20140912
20150524
20151106
20160130
20161014
20170213
20170621
20180526
20200109
20200625
20210819
Twitter | Community Guidelines
20090918
20091025
20100301
20100323
20101004
20101020
20110124
20110412
20110902
20110903
20120510
20120919
20121029
20130530
20130813
2

In [ ]:
os.listdir('./')

['.config', 'drive', '.ipynb_checkpoints', 'platforms.json', 'sample_data']

In [ ]:
with open('platforms.json', 'r') as input:
  data = json.load(input)

FileNotFoundError: ignored

In [ ]:
companies = ['Facebook', 'Instagram', 'Twitter', 'YouTube']
policies = ['Community Guidelines', 'Privacy Policy', 'Terms of Service']

In [ ]:
for c in companies:
  for p in policies:
    try:
    num_docs = len(os.listdir('/content/drive/MyDrive/Code/HIIG/pgArchive/data/diffs/'+c+'/'+p))
    num_docs = (num_docs*2)-1
    print(c + ': '+p + ' '+str(num_docs))

Facebook: Community Guidelines 863
Facebook: Privacy Policy 1723
Facebook: Terms of Service 2599


FileNotFoundError: ignored